In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import logging

from inference import infer_tool
from inference.infer_tool import Svc
import torch

# play audio using IPython.display.Audio
from IPython.display import Audio
from IPython.display import display
import utils
import random
import soundfile


logging.getLogger("numba").setLevel(logging.WARNING)
chunks_dict = infer_tool.read_temp("inference/chunks_temp.json")


feature_retrieval = True

if feature_retrieval:
    cluster_model_path = ""
else:
    cluster_model_path = "logs/22k/kmeans_10000.pt"


sovits_model_path = "/home/alexander/Projekte/so-vits-svc/logs/22k_cfm/G_ema_192000.pth"
sovits_config_path = "/home/alexander/Projekte/so-vits-svc/logs/22k_cfm/config.json"

device = "cpu"

svc_model = Svc(
    net_g_path=sovits_model_path,
    config_path=sovits_config_path,
    cluster_model_path=cluster_model_path,
    speaker_encoder_path="/home/alexander/Projekte/SpeakerEncoder/tb_logs/lightning_logs/version_15/checkpoints/checkpoint.ckpt",
    feature_retrieval=feature_retrieval,
    device=device,
)


hps = utils.get_hparams_from_file(sovits_config_path, True)
speaker_dict = {v: k for k, v in hps.spk.items()}

speaker_files = {}

with open(
    "/home/alexander/Projekte/so-vits-svc/filelists/gametts_train.txt", "r"
) as rf:
    for line in rf:
        wav_path, spk = line.split("|")
        spk = spk.strip()
        speaker_name = speaker_dict[int(spk)]

        if speaker_name not in speaker_files:
            speaker_files[speaker_name] = []

        speaker_files[speaker_name].append(wav_path)


def inference(
    raw_audio_path,
    target_audio_path,
    target_speaker=None,
    cluster_infer_ratio=0,
    n_timesteps=2,
    guidance_scale=0.0,
    temperature=1.0,
    f0_predictor="crepe",
    solver="euler",
):
    trans = 0
    f0_filter_threshold = 0.05

    infer_tool.mkdir(["raw", "results"])

    infer_tool.format_wav(raw_audio_path)
    for target_wav in target_audio_path:
        infer_tool.format_wav(target_wav)

    kwarg = {
        "raw_audio_path": raw_audio_path,
        "raw_target_audio_path": target_audio_path,
        "target_speaker": target_speaker,
        "tran": trans,
        "cluster_infer_ratio": cluster_infer_ratio,
        "n_timesteps": n_timesteps,
        "f0_predictor": f0_predictor,
        "cr_threshold": f0_filter_threshold,
        "temperature": temperature,
        "guidance_scale": guidance_scale,
        "solver": solver,
    }
    mel = svc_model.slice_inference(**kwarg)


    return mel

In [ ]:
from vdecoder.IstftGenerator.model import Generator
from vdecoder.utils import AttrDict

import json


model_file = "/home/alexander/Projekte/iSTFT-Avocodo-pytorch/cp_resnet/g_00400000"
config_file = "/home/alexander/Projekte/iSTFT-Avocodo-pytorch/cp_resnet/config.json"
with open(config_file) as f:
    data = f.read()
json_config = json.loads(data)
h = AttrDict(json_config)

generator = Generator(input_channels=h.num_mels, num_layers=8).to(device)

checkpoint_dict = torch.load(model_file, map_location=device)
generator.load_state_dict(checkpoint_dict["generator"])

generator.eval()
generator.remove_weight_norm()

In [ ]:
import time
from glob import glob

# /home/alexander/Projekte/vits-emotts/samples/Test.wav
source_audio = "/home/alexander/Projekte/so-vits-svc/samples/4.wav"
target_speaker = "Inquisitor"
target_audios = speaker_files[target_speaker]
target_audios = random.sample(target_audios, len(target_audios[:10]))

# target_audios = glob("/home/alexander/Projekte/so-vits-svc/samples/Suchender/*.wav", recursive=True)
# target_audios = random.sample(target_audios, len(target_audios[:10]))

# target_audios = ["/home/alexander/Projekte/so-vits-svc/samples/2.wav", "/home/alexander/Projekte/so-vits-svc/samples/3.wav", "/home/alexander/Projekte/so-vits-svc/samples/4.wav"]
# target_audios = random.sample(target_audios, len(target_audios[:10]))

print("Speaker:", target_speaker)
print("Source Audio:")
display(Audio(source_audio, rate=svc_model.target_sample, autoplay=False))
print(f"Target Audio ({len(target_audios)})")
display(Audio(target_audios[0], rate=svc_model.target_sample, autoplay=False))

start_time = time.time()

mel = inference(
    source_audio,
    target_audios,
    target_speaker=target_speaker,
    f0_predictor="rmvpe",
    cluster_infer_ratio=0.7,
    n_timesteps=16,
    temperature=1.0,
    guidance_scale=0.5,
    solver="rk4",
)

print(f"VC Time: {time.time() - start_time}")

start_time = time.time()

mel = mel.clone()

audio = generator(mel).data.cpu().float().numpy()

print(f"Decoder Time: {time.time() - start_time}")


soundfile.write(
    f"/home/alexander/Projekte/so-vits-svc/raw/{target_speaker.lower()}.wav",
    audio,
    44100,
    format="WAV",
    subtype="PCM_16",
)

display(Audio(audio, rate=44100, autoplay=False))

In [ ]:
utils.plot_spectrogram_to_numpy(mel[0].cpu().numpy(), return_figure=True)[1]

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from modules.mel_processing import mel_spectrogram_torch
import utils
import torchaudio

audio, sr = torchaudio.load(source_audio)
mel_gen = mel_spectrogram_torch(audio, 1024, 80, 22050, 256, 1024, 0, None)
utils.plot_spectrogram_to_numpy(mel_gen[0].cpu().numpy(), return_figure=True)[1]

In [ ]:
import time
from glob import glob

# /home/alexander/Projekte/vits-emotts/samples/Test.wav
source_audio = "/home/alexander/Projekte/so-vits-svc/samples/Eminem - Rap God.wav"

speakers = [
    "Fletcher_g1",
    "Thorus_g1",
    "Inquisitor",
    "Vatras_g2",
    "Gorn_g2",
    "Diego_g1",
    "Halvor_g2",
    "Ulthar_g2",
    "Xardas_g2",
    "Milten_g1",
    "Lester_g1",
    "Player_g3",
    "Zuben_g3",
    "Grompel_g3",
]
for target_speaker in speakers:
    # target_speaker = random.choice(list(speaker_files.keys())) #"Vatras_g2"
    target_audios = speaker_files[target_speaker]
    target_audios = random.sample(target_audios, len(target_audios[:10]))

    # target_audios = glob("/home/alexander/Projekte/so-vits-svc/samples/Suchender/*.wav", recursive=True)
    # target_audios = random.sample(target_audios, len(target_audios[:10]))

    # target_audios = ["/home/alexander/Projekte/so-vits-svc/samples/2.wav", "/home/alexander/Projekte/so-vits-svc/samples/3.wav", "/home/alexander/Projekte/so-vits-svc/samples/4.wav"]
    # target_audios = random.sample(target_audios, len(target_audios[:10]))

    print("Speaker:", target_speaker)
    print(f"Target Audio ({len(target_audios)})")
    display(Audio(target_audios[0], rate=svc_model.target_sample, autoplay=False))

    start_time = time.time()

    mel = inference(
        source_audio,
        target_audios,
        target_speaker=target_speaker,
        f0_predictor="rmvpe",
        cluster_infer_ratio=0.7,
        n_timesteps=15,
        temperature=1.0,
        guidance_scale=0.5,
        solver="midpoint",
    )

    print(f"VC Time: {time.time() - start_time}")

    start_time = time.time()

    mel = mel.clone()

    audio = generator(mel).data.squeeze(0).cpu().float().numpy()

    print(f"Decoder Time: {time.time() - start_time}")

    soundfile.write(
        f"/home/alexander/Projekte/so-vits-svc/raw/{target_speaker.lower()}.wav",
        audio,
        44100,
        format="WAV",
        subtype="PCM_16",
    )

    # # numpy_audio = audio.squeeze(0).cpu().detach().numpy()
    # # audio = generator(mel_spec).squeeze(0).cpu().detach().numpy()
    # display(Audio(audio, rate=44100, autoplay=False))

## Testing

In [ ]:
augmented_audio = torch.from_numpy(audio)
mel = mel_spectrogram_torch(augmented_audio, 2048, 80, 44100, 512, 2048, 0, 11000)
utils.plot_spectrogram_to_numpy(mel[0].cpu().numpy(), return_figure=True)[1]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class FeatureShiftModel(nn.Module):
    def __init__(
        self, feature_dim=192, num_latents=32, cluster_infer_ratio=0.5, latent_dim=128
    ):
        super(FeatureShiftModel, self).__init__()
        self.feature_dim = feature_dim
        self.num_latents = num_latents
        self.cluster_infer_ratio = cluster_infer_ratio
        self.latent_dim = latent_dim

        # Encoder: Source features to latent space
        self.encoder = nn.Sequential(
            nn.Linear(feature_dim, latent_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(latent_dim, latent_dim),
        )

        # Decoder: Latent space (combined with target latents) to shifted features
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + feature_dim, feature_dim),  # Assuming concatenation
            nn.LeakyReLU(0.2),
            nn.Linear(feature_dim, feature_dim),
        )

    def forward(self, source_features, target_latents):
        # Encode source features
        # Assuming source_features is of shape (batch, feature_dim, sequence_length)
        # Need to permute to apply linear layer: (batch, sequence_length, feature_dim)
        source_features_permuted = source_features.permute(0, 2, 1)
        encoded_features = self.encoder(source_features_permuted)

        # Aggregate target latents
        aggregated_target_latents = target_latents.mean(dim=2, keepdim=True)
        expanded_target_latents = aggregated_target_latents.expand(
            -1, -1, source_features.shape[2]
        )

        # Combine encoded source features with target latents
        # For simplicity, we concatenate along the feature dimension
        combined = torch.cat(
            (encoded_features, expanded_target_latents.permute(0, 2, 1)), dim=-1
        )

        # Decode the combined representation to get shifted features
        shifted_features = self.decoder(combined)

        shifted_features = shifted_features.permute(
            0, 2, 1
        )  # Permute back to original shape

        # Combine decoded features with original source features (optional)
        shifted_features = (
            self.cluster_infer_ratio * shifted_features
            + (1 - self.cluster_infer_ratio) * source_features
        )

        return shifted_features


# Example usage
batch_size = 4
sequence_length = 182
cluster_infer_ratio = 0.5

model = FeatureShiftModel(
    feature_dim=192,
    num_latents=32,
    cluster_infer_ratio=cluster_infer_ratio,
    latent_dim=128,
)
source_features = torch.randn(batch_size, 192, sequence_length)
target_latents = torch.randn(batch_size, 192, 32)  # Target latents

shifted_features = model(source_features, target_latents)
print(shifted_features.shape)  # Expected shape: (batch, 192, sequence length)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import modules.attentions as attentions
from modules.feature_encoder_decoder import FeatureEncoderDecoder

# Example usage
batch_size = 4
sequence_length = 182

model = FeatureEncoderDecoder(hidden_channels=192)
x = torch.randn(batch_size, 192, sequence_length)  # Adjusted shape for LSTM
x_mask = torch.ones(batch_size, 1, sequence_length)
prosody_latent = torch.randn(batch_size, 192, 32)  # Target latents
prosody_mask = torch.ones(batch_size, 1, 32)

shifted_features = model(
    x, x_mask, prosody_latent, prosody_mask
)  # Transpose back to original shape
print(shifted_features.shape)  # Expected shape: (batch, sequence_length, 192)